| Field      | Units| Code | Frequency| Source  |Derivation Notes|
| :---        |    :----:   |    :---: |
|Total Debt - gross external debt as % of GDP|	%|	GGXWDG_NGDP|	Q|	QEDS|	
|Short-Term Debt - gross short-term external debt as % of GDP|	%|	DT.DOD.DSTC.ZS|	Q|	QEDS|	
|Real Interest Rate - lending interest rate adjusted for inflation|	%|	FILR_PA|	A/Q/M|	IMF - IFS|	To Derive - Subtract Inflation
|Foreign Exchange Reserves - total reserves (without gold) as % of GDP|	%|	RAXGFX_USD|	A/Q/M|	IMF - IFS|	To Derive
|Trade Openness - ratio of exports plus imports to GDP|	Unitless|	(TXG_FOB_USD + TMG_CIF_USD)/GDP|	Q|	IMF - IFS|	
|Imports - imports of goods and services at current prices in USD|	USD|	TMG_CIF_USD|	A/Q/M|	IMF - IFS|	
|Exports - exports of goods and services at current prices in USD|	USD|	TXG_FOB_USD|	A/Q/M|	IMF - IFS|	
|Current Account - current account balance as % of GDP|	%|	BCAXF_BP6_USD|	A/Q|	IMF - BOP|	
|Portfolio Investments - portfolio investment net at current USD|	USD|	IAP_BP6_USD|	A/Q|	IMF - IFS|	
|FDI - net FDI inflows as % of GDP|	USD|	BFDLXF_BP6_USD|	A/Q|	IMF - BOP|	To Derive
|Real GDP - annual real GDP at current USD|	USD|	NY.GDP.MKTP.CD|	A|	WDI|	To Derive: Will have to divide by 4 for Q, 12 for M, 365 for D
|Real GDP Growth - annual growth of real GDP|	|	just make it - WILL THIS BE USEFUL?!|	|	|	Not Required
|Inflation - rate of change in CPI|	%|	PCPI_IX|	A/Q/M|	IMF - CPI|	
|M2 - Money plus quasi-money, Domestic Currency|	XDC|	35L___XDC|	A/Q/M|	IMF - IFS|	used in derivaion
|M2 Multiplier Growth - annual growth of M2|	%|	FM.LBL.BMNY.ZG|	A|	WDI|	To Derive M2
|Reserves - Total Reserves excluding Gold, US Dollars|	USD|	RAXG_USD|	A/Q/M|	IMF - IFS|	used in derivaion
|M2/Reserves - ratio of M2 to foreign exchange reserves|	XCD*FX/USD|	FM.LBL.BMNY.IR.ZS(FM.LBL.BMNY.IR.ZS_DS2)|	A|	WDI|	
|REER Overall - deviation of real effective exchange rate from 5 year rolling mean|	Unitless|	EREER_IX|	A/Q/M|	WDI|	Need to derived the 5 year period average 
|Government Spending - general government final spending as % of GDP|	%|	GGX_NGDP|	A|	World Bank|	Need to Take it from the CVS Annually
|Fixed Capital Formation - gross fixed capital formation at current USD|	USD|	NE.GDI.FTOT.CD|	A|	WDI|	
|Unemployment - unemployment total as % of total labor force|	%|	LUR_PT|	A/Q|	IMF - IFS|	
|Contagion - event of a currency crisis in any country of the same region (t − 1)|	Indicator|	|	|	Derived|	
|Domestic Credit - ratio of domestic credit to GDP|	%/(USD)|	32____XDC|	A/Q/M|	IMF - IFS|	Divide by GDP
|Lending Interest Rate - the bank rate meets the short- and medium-term financing needs|	%|	FILR_PA|	A/Q/M|	IMF - IFS|	
|Deposit Interest Rate - rate paid by banks for demand, time, or savings deposits|	%|	FIDR_PA|	A/Q/M|	IMF - IFS|	




In [9]:
!pip install -q -r requirements.txt
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler


import datapackage
from functools import reduce
import world_bank_data as wb
import requests
# Plotting libraries
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
#import tabula
#from tabula import read_pdf

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


# Data Cleaning function

In [10]:
def clean_wdi_data(dataset, countries, date_from = 1980, date_to = 2002 ):
    '''
    Loads WOE CSV 
    dataset {df}: WDI Data sample
    countries {List}: List of countries to be inclided in Dataset
    date_from {int}: Lowest year - default 1980
    date_to {int}: Highest Yeat - default 2020

    Output{df}: Year, Country Name , Country Code, [Series Names]

    '''
    #Clean titles
    #Create Dictionary of column header mapping & use rename function
    old_columns = dataset.columns[4::]
    yr_columns = dataset.columns[4::].str[:4]
    yr_columns_dict = dict(zip(old_columns, yr_columns))
    dataset.rename(columns = yr_columns_dict, inplace = True)
    
    #Country Selection
    dataset = dataset[dataset['Country Name'].isin(countries)].copy()
    
    #Reform Pivotted Data
    dataset.drop([ 'Series Code'], axis=1, inplace = True)
    dataset = dataset.melt(id_vars=['Country Name','Country Code','Series Name'], var_name='Year', value_name='Value' ).copy()
    dataset = dataset.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Series Name', values='Value').reset_index().copy()
    dataset.rename(columns = {'Country Code':'ISO Code'}, inplace = True)

    return dataset

def clean_IFS_data(dataset, countries, frequency, date_from, Series_Codes , date_to):
    '''
    Loads IFS CSV 
    dataset {df}: WDI Data sample
    countries {List}: List of countries to be inclided in Dataset
    date_from {int}: Lowest year - default 1980
    date_to {int}: Highest Yeat - default 2020

    Output{df}: 'Year', 'Country Name', 'Country Code', [Series Names]

    '''
    #Country Selection
    dataset = dataset[dataset['Country Name'].isin(countries)].copy()   

    #Concat Base year & Drop Base Year Column
    dataset['Indicator Name'] = np.where(dataset['Base Year'].isnull(),dataset['Indicator Name'], dataset['Indicator Name'] + " (" + dataset['Base Year'] +")").copy()
    
    # Filter by then concat the Indicator Code
    dataset = dataset[dataset['Indicator Code'].isin(Series_Codes)].copy()
    dataset['Indicator Name'] = dataset['Indicator Name'] + " - " + dataset['Indicator Code']

    dataset = dataset[dataset['Attribute']!='Status'].copy()
    
    dataset.drop(['Base Year', 'Indicator Code', 'Attribute'], axis=1, inplace = True)


    #Melt Data 
    dataset = dataset.melt(id_vars=['Country Name','Country Code','Indicator Name'], var_name='Year', value_name='Value' ).copy()

    #Frequency - Annual - Done
    if frequency == 'Y':
        non_ann = ['M', 'Q']
        dataset = dataset[~dataset.Year.str.contains('|'.join(non_ann))].copy()
    elif frequency == 'Q':
        dataset = dataset[dataset.Year.str.contains('Q')].copy()
    elif frequency == 'M':
        dataset = dataset[dataset.Year.str.contains('M')].copy()

    #Pivot Indicators to Columns
    dataset['Value'] = dataset['Value'].astype(float)
    dataset = dataset.groupby(['Year', 'Country Name', 'Country Code', 'Indicator Name']).agg({'Value':'sum'}).reset_index().copy()
    dataset = dataset.pivot(index=['Year', 'Country Name', 'Country Code'], columns='Indicator Name', values='Value').reset_index().copy()
    dataset.rename(columns = {'Country Code':'IMF Code'}, inplace = True)

    return dataset


# Generic API Functions

## Single Column Calls

In [120]:
def Call_IMF(DB,freq,Country_iso2, start, finish, Indicator_code):
    '''
    Returns "Year" & "Values" of series with column names after the IMF Code
    '''
    core = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/'
    key = f'{DB}/{freq}.{Country_iso2}.{Indicator_code}'
    time = f'.?startPeriod={start}&endPeriod={finish}'
    url = f'{core}{key}{time}'

    # Get data from the above URL using the requests package
    data = requests.get(url).json()

    # Load data into a pandas dataframe
    series = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])
    series['@OBS_VALUE'] = series['@OBS_VALUE'].astype(float)
    series.rename(columns = {'@TIME_PERIOD':'Year', '@OBS_VALUE':Indicator_code}, inplace = True)
    return series

def Call_WB(iso3, id,start, finish):
    '''
    Returns "Country"(ISO3), "Year" & "Values" of series with column names after the World Bank Code
    '''
    daterange = f'{start}:{finish}'
    Data = pd.DataFrame(wb.get_series(id, date=daterange, id_or_value='id', simplify_index=True)).reset_index()
    Data = Data[Data['Country']==iso3]
    return Data


def Get_Monthly_GDP(iso3,start, finish):
    '''
    Returns Monthly Call_WB()
    '''
    Data = Call_WB(iso3=iso3, id='NY.GDP.MKTP.CD',start=start, finish=(finish+1))
    Data['Year'] = pd.to_datetime(Data['Year'], exact = False, format='%Y%')
    Data.set_index(['Year'],inplace=True)
    Data.drop(columns='Country', inplace = True)
    Data = Data.resample('MS').ffill() / 12
    Data = Data.reset_index()

    #trail = pd.date_range(start=f'01/02/{finish}', periods=11, freq = 'MS')
    #Add_on = pd.DataFrame({'Year':trail})
    #Add_on.set_index(['Year'],inplace=True)
    #Add_on['NY.GDP.MKTP.CD'] =  Data['NY.GDP.MKTP.CD'].tail(1).item()
#
    #Data = pd.concat([Data,Add_on])
    #Data = Data.reset_index()
    return Data




## Field Specific Functions

In [ ]:
mth_fields = ['FILR_PA','RAXGFX_USD','TMG_CIF_USD','TXG_FOB_USD','PCPI_IX',
                '35L___XDC','RAXG_USD','EREER_IX','32____XDC','FILR_PA','FIDR_PA']
"+".join(mth_fields) 

'FILR_PA+RAXGFX_USD+TMG_CIF_USD+TXG_FOB_USD+PCPI_IX+35L___XDC+RAXG_USD+EREER_IX+32____XDC+FILR_PA+FIDR_PA'

In [126]:
def Call_M_IMF(iso2, start, finish):
    mth_fields = ['RAXGFX_USD','TMG_CIF_USD','TXG_FOB_USD','PCPI_IX',
                '35L___XDC','RAXG_USD','EREER_IX','32____XDC','FILR_PA','FIDR_PA','ENDA_XDC_USD_RATE']
    
    call_string = "+".join(mth_fields)    
    core = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/'
    key = f'IFS/M.{iso2}.{call_string}'
    time = f'.?startPeriod={start}&endPeriod={finish}'
    url = f'{core}{key}{time}'

    # Get data from the above URL using the requests package
    data = requests.get(url).json()
    # Load data into a pandas dataframe
    #series = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])
    appended_data = []
    num_rows = pd.DataFrame(data['CompactData']['DataSet']['Series']).shape[0]
    print(pd.DataFrame(data['CompactData']['DataSet']['Series']))
    for c in range(num_rows):
        header = data['CompactData']['DataSet']['Series'][c]['@INDICATOR']
        print(header)
        unit_mult = float(data['CompactData']['DataSet']['Series'][c]['@UNIT_MULT'])
        series = pd.DataFrame(data['CompactData']['DataSet']['Series'][c]['Obs'])
        series['@OBS_VALUE']=series['@OBS_VALUE'].astype(float)
        if unit_mult != 0:
            series['@OBS_VALUE'] = series['@OBS_VALUE']*(10**unit_mult)

        series.rename(columns = {'@TIME_PERIOD':'Year', '@OBS_VALUE':header}, inplace = True)
        series['Year'] = pd.to_datetime(series['Year'], exact = False, format='%Y%')
        appended_data.append(series)

    #Add GDP Data
    WB_countries = wb.get_countries().reset_index()
    iso3 = WB_countries['id'][WB_countries['iso2Code']==iso2].item()
    appended_data.append(Get_Monthly_GDP(iso3,start, finish))
    merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['Year'],how='outer'), appended_data).fillna(np.NaN)

    #Add Derived Monthly GDP Date
    

    #Create Derived Fields
    #Real interest rate adjusted to inflation
    if 'FILR_PA' in merged_df.columns and 'PCPI_IX' in merged_df.columns:
        merged_df['Real Interest Rate(%)'] = merged_df['FILR_PA'] - merged_df['PCPI_IX']
    #total reserves (without gold) as % of GDP
    if 'ENDA_XDC_USD_RATE' in merged_df.columns and 'RAXGFX_USD' in merged_df.columns and 'NY.GDP.MKTP.CD' in merged_df.columns:
        merged_df['Foreign Exchange Reserves(%)'] = 100*(merged_df['RAXGFX_USD']/ (merged_df['NY.GDP.MKTP.CD']*merged_df['ENDA_XDC_USD_RATE']))
    #M2 Multiplier Growth
    if '35L___XDC' in merged_df.columns:
        merged_df['M2 Multiplier Growth (%)'] = 100*(merged_df['35L___XDC'].pct_change())
    #'M2/Reserves'
    if '35L___XDC' in merged_df.columns and 'RAXGFX_USD' in merged_df.columns and 'ENDA_XDC_USD_RATE' in merged_df.columns:
        merged_df['M2/Reserves'] = merged_df['35L___XDC']/(merged_df['RAXGFX_USD']*merged_df['ENDA_XDC_USD_RATE'])
    #1 year Deviation of REED
    if 'EREER_IX' in merged_df.columns:
        merged_df['REED 12mth std dev'] = merged_df['EREER_IX'].rolling(12).std()
    #Domestic Credit to GDP
    if '32____XDC' in merged_df.columns and 'NY.GDP.MKTP.CD' in merged_df.columns and 'ENDA_XDC_USD_RATE' in merged_df.columns:
        merged_df['Domestic Credit to GDP'] = (merged_df['32____XDC']/(merged_df['NY.GDP.MKTP.CD']*merged_df['ENDA_XDC_USD_RATE']))

    to_drop = ['RAXGFX_USD','35L___XDC','RAXG_USD','EREER_IX','32____XDC','@OBS_STATUS']
    filtered_list = [col for col in to_drop if col  in merged_df.columns.to_list() ]
    merged_df.drop(columns =filtered_list, inplace = True )

    merged_df['days in month']  = pd.to_datetime(merged_df['Year']).dt.to_period('M').dt.days_in_month
    
    merged_df.set_index('Year', inplace = True)
    merged_df = merged_df.resample('D').pad()
    split = ['TMG_CIF_USD','TXG_FOB_USD','M2 Multiplier Growth']

    for i in range(len(split)):
        if split[i] in merged_df.columns.to_list():
            merged_df[split[i]] = merged_df[split[i]] / merged_df['days in month']


    return merged_df

In [130]:
MEX_FULL = Call_M_IMF('MX', 1980,2020)


   @FREQ @REF_AREA         @INDICATOR @UNIT_MULT @TIME_FORMAT  \
0      M        MX        TMG_CIF_USD          6          P1M   
1      M        MX         RAXGFX_USD          6          P1M   
2      M        MX           RAXG_USD          6          P1M   
3      M        MX  ENDA_XDC_USD_RATE          0          P1M   
4      M        MX            PCPI_IX          0          P1M   
5      M        MX           EREER_IX          0          P1M   
6      M        MX        TXG_FOB_USD          6          P1M   
7      M        MX          35L___XDC          6          P1M   
8      M        MX          32____XDC          6          P1M   
9      M        MX            FIDR_PA          0          P1M   
10     M        MX            FILR_PA          0          P1M   

                                                  Obs @BASE_YEAR  
0   [{'@TIME_PERIOD': '1980-01', '@OBS_VALUE': '13...        NaN  
1   [{'@TIME_PERIOD': '1980-01', '@OBS_VALUE': '19...        NaN  
2   [{'@TIME_PERIO

In [128]:
MEX_CUT = MEX_FULL.copy()
#MEX_CUT = MEX_CUT.head(30)
#MEX_CUT.set_index('Year', inplace = True)
#MEX_CUT = MEX_CUT.resample('D').pad()


In [124]:
Get_Monthly_GDP('MEX',1999, 2018)

,Year,NY.GDP.MKTP.CD
0,1999-01-01,5.001941e+10
1,1999-02-01,5.001941e+10
2,1999-03-01,5.001941e+10
3,1999-04-01,5.001941e+10
4,1999-05-01,5.001941e+10
...,...,...
236,2018-09-01,1.018624e+11
237,2018-10-01,1.018624e+11
238,2018-11-01,1.018624e+11
239,2018-12-01,1.018624e+11


In [132]:
MEX_CUT.shape

(7306, 14)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5afecb5f-7d28-4b37-ae51-8d94c0fa9663' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>